In [30]:
from playwright.async_api import async_playwright

playwright = await async_playwright().start()
browser = await playwright.chromium.connect_over_cdp("http://localhost:9222")
default_context = browser.contexts[0]

# Retrieve the first page in the context.
page = default_context.pages[0]
page.set_default_timeout(5000)

In [31]:
await page.title()

'Contact information'

In [32]:
# gemini_page = None
# for tab in default_context.pages:
#     title = await tab.title()
#     print(title)
#     if 'Google AI' in title:
#         gemini_page = tab
# if not gemini_page:
#     raise Exception('Gemini Page not found')

In [36]:
import requests

import re

def extract_first_python_code(markdown_text: str):
    # Pattern to match the first ```python ``` code block
    pattern = r"```python(.*?)```"

    # Using re.DOTALL to make '.' match also newlines
    match = re.search(pattern, markdown_text, re.DOTALL)
    if match:
        # Return the first matched group, which is the code inside the ```python ```
        return match.group(1).strip()
    else:
        # Return None if no match is found
        return None

class RemoteActionEngine:
    def __init__(self, url) -> None:
        self.url = url
        
    def get_action_ground_truth(self, query: str, html: str) -> str:
        data = {
            "query": query,
            "HTML": html
        }
        endpoint = self.url + "/process_direct"
        response = requests.post(endpoint, json=data)

        if response.status_code == 200:
            code = extract_first_python_code(response.json())
            return code
        else:
            raise Exception("Failed to retrieve data, status code:", response.status_code)


    def get_action(self, query: str, html: str) -> str:
        data = {
            "query": query,
            "HTML": html
        }
        url = self.url + "/process"
        response = requests.post(url, json=data)
        
        if response.status_code == 200:
            response_data = response.json()
            code, source_nodes = response_data['code'], response_data['retrieved_nodes']
            return code, source_nodes
        else:
            raise Exception("Failed to retrieve data, status code:", response.status_code)
        
    def get_nodes(self, query: str, html: str) -> str:
        data = {
            "query": query,
            "HTML": html
        }
        url = self.url + "/get_index"
        response = requests.post(url, json=data)
        
        if response.status_code == 200:
            response_data = response.json()
            code, source_nodes = response_data['code'], response_data['retrieved_nodes']
            return code, source_nodes
        else:
            raise Exception("Failed to retrieve data, status code:", response.status_code)
        
    
    def process_fat_node(self, query: str, html: str):
        data = {
            "query": query,
            "HTML": html
        }
        url = self.url + "/process_fat_node"
        response = requests.post(url, json=data)
        
        if response.status_code == 200:
            response_data = response.json()
            code, source_nodes, metadata = response_data['code'], response_data['retrieved_nodes'], response_data['metadata']
            return code, source_nodes, metadata
        else:
            raise Exception("Failed to retrieve data, status code:", response.status_code)

In [37]:
url = "http://127.0.0.1:8000"
remote_action_engine = RemoteActionEngine(url)

In [38]:
from IPython.display import display, HTML
from bs4 import BeautifulSoup

def beautify_html(html_string):
    # Use BeautifulSoup to prettify the HTML
    soup = BeautifulSoup(html_string, 'html.parser')
    pretty_html_string = soup.prettify()
    return pretty_html_string


In [39]:
import os
from llama_index.llms.azure_openai import AzureOpenAI

api_key=os.getenv("AZURE_OPENAI_KEY")
api_version="2023-05-15"
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
model = "gpt-4"
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4-turbo")

class LLM(AzureOpenAI):
    def __init__(self):
        super().__init__(
            model=model,
            deployment_name=deployment_name,
            api_key=api_key,
            azure_endpoint=azure_endpoint,
            api_version=api_version,
            temperature=0.0
        )
llm = LLM()

### Classification prompt

In [40]:
CLASSIFICATION_PROMPT = '''
Your goal is to assesss if for a given user query and HTML associated, the HTML contains the elements that are required to perform the action specified in the query.

To assess it, look at the presence of User-Facing Attributes, text selectors, like text="Visible Text", that could be used to uniquely identify the element requested in the query. 
You can also use Attributes like aria-label, aria-labelledby, role, etc., to target elements.
When user-facing attributes are not available or sufficient, Prefer class names and IDs that are meaningful and unlikely to change. 
Look at parent-child relationships to narrow down the element, especially when looking for elements within a specific section of the page

Answers are provided in two fields, 'Thought', which contains bullet points explaining the though process and 'Identifiable', which is a boolean indicating if the element can be identified based on the query.
---

HTML:
<div class="oyXaNc" jsaction="rcuQ6b:vZc4S;O22p3e:zjh6rb;b2trFe:eVidQc;JIbuQc:RgMCxe(YlCLKb);sPvj8e:d3sQLd;TYy3Ne:RgMCxe;" jscontroller="UmOCme" jsname="GCYh9b">
 <input jsname="DTMEae" name="entry.839337160_sentinel" type="hidden"/>
 <form action="https://docs.google.com/forms/u/0/d/e/1FAIpQLSfvfEMQPG9rMVH8p_Q_PnkffTIccd1g_VJGWmTpId5zYVhzVg/formResponse" data-clean-viewform-url="https://docs.google.com/forms/d/e/1FAIpQLSfvfEMQPG9rMVH8p_Q_PnkffTIccd1g_VJGWmTpId5zYVhzVg/viewform" data-dlp-data="%.@.null,false]" data-draft-response='%.@."",1711844705913,1711907458489,null,"8112371325920032428",null,[[null,null,["31-40"],false,null,null,839337160],[null,null,["+123"],false,null,null,1166974658],[null,null,["Daniel"],false,null,null,2005620554]]]' data-first-entry="0" data-is-first-page="true" data-last-entry="4" data-response="%.@.]" data-shuffle-seed="8112371325920032428" id="mG61Hd" jsmodel="TOfxwf Q91hve CEkLOc" method="POST" target="_self">
  <div jsaction="rcuQ6b:rcuQ6b;UxRBlf:rcuQ6b;" jscontroller="yUS4Lc">
   <div jsname="o6bZLc">
    <input name="entry.839337160" type="hidden" value="31-40"/>
    <input name="entry.1166974658" type="hidden" value="+123"/>
    <input name="entry.2005620554" type="hidden" value="Daniel"/>
    <input name="entry.1343513443" type="hidden" value=""/>
    <input name="dlut" type="hidden" value="1711907458489"/>
   </div>
  </div>
  <div class="o3Dpx" role="list">
   <div class="Qr7Oae" role="listitem">
    <div data-params='%.@.[1633920210,"Name","This is where you fill name",0,[[2005620554,null,true,null,null,null,null,null,null,null,[]]],null,null,null,null,null,null,[null,"Name"],[null,"This is where you fill name"]],"i1","i2","i3",false]' jsmodel="CP1oW">
     <div class="geS5n" jsaction="EEvAHc:yfX9oc;" jscontroller="sWGJ4b" jsname="WsjYwc">
      <div class="z12JJ">
       <div class="M4DNQ">
        <div aria-describedby="i4" aria-level="3" class="HoXoMd D1wxyf RjsPE" id="i1" role="heading">
         <span class="M7eMe">
          Name
         </span>
         <span aria-label="Required question" class="vnumgf" id="i4">
          *
         </span>
        </div>
        <div class="gubaDc OIC90c RjsPE" id="i2">
         This is where you fill name
        </div>
       </div>
      </div>
      <div class="AgroKb" jsaction="rcuQ6b:vZc4S;O22p3e:zjh6rb;b2trFe:eVidQc;sPvj8e:d3sQLd;AHmuwe:h06R8;" jscontroller="oCiKKc">
       <div class="rFrNMe k3kHxc RdH0ib yqQS1 zKHdkd CDELXb" jsaction="clickonly:KjsqPd; focus:Jt1EX; blur:fpfTEe; input:Lg5SV" jscontroller="pxq3x" jsname="YPqjbf" jsshadow="">
        <div class="aCsJod oJeWuf">
         <div class="aXBtI Wic03c">
          <div class="Xb9hP">
           <input aria-describedby="i2 i3" aria-disabled="false" aria-labelledby="i1" autocomplete="off" badinput="false" class="whsOnd zHQkBf" data-initial-dir="auto" data-initial-value="Daniel" dir="auto" jsname="YPqjbf" required="" tabindex="0" type="text"/>
           <div aria-hidden="true" class="ndJi5d snByac" jsname="LwH6nd">
            Your answer
           </div>
          </div>
          <div class="i9lrp mIZh1c">
          </div>
          <div class="OabDMe cXrdqd" jsname="XmnwAc">
          </div>
         </div>
        </div>
        <div class="LXRPh">
         <div class="ovnfwe Is7Fhb" jsname="ty6ygf">
         </div>
        </div>
       </div>
      </div>
      <div class="SL4Sz" id="i3" jsname="Rfh2Tc" role="alert">
      </div>
     </div>
    </div>
   </div>
  </div>
 </form>
</div>

Query: Select the input with placeholder 'Your answer' below 'Name', then type 'Daniel'
Thought: 
- First we need to identify the input field with the placeholder 'Your answer' that is below the 'Name' label.	
- We can use the aria-labelledby attribute to find the input field associated with the 'Name' label.
- The 'Name' label has an id of "i1", so we can use that to find the input field.
Identifiable: Yes

---

HTML:
iv class="Jwjdfe"><div class="dMALK" jsname="YlCLKb"><div role="button" class="uArJ5e UQuaGc kCyAyd" jscontroller="VXdfxd" jsaction="click:cOuCgd; mousedown:UX7yZ; mouseup:lbsD7e; mouseenter:tfO1Yc; mouseleave:JywGue;touchstart:p6p2H; touchmove:FwuNnf; touchend:yfqBxc(preventDefault=true); touchcancel:JMtRjd;focus:AHmuwe; blur:O22p3e; contextmenu:mg9Pef;" jsshadow="" tabindex="0"><div class="Fvio9d MbhUzd" jsname="ksKsZd"></div><div class="e19J0b CeoRYc"></div><span jsslot="" class="l4V7wb Fxmcue"><span class="NPEfkd RveJvd snByac">Clear selection</span></span></div></div></div></div><d
iv jsmodel="CP1oW" data-params="%.@.[1846923513,&quot;Age&quot;,null,2,[[839337160,[[&quot;18-24&quot;,null,null,null,false],[&quot;25-30&quot;,null,null,null,false],[&quot;31-40&quot;,null,null,null,false],[&quot;41-50&quot;,null,null,null,false],[&quot;51&quot;,null,null,null,false],[&quot;&quot;,null,null,null,true]],false,null,null,null,null,null,false,null,[]]],null,null,null,null,null,null,[null,&quot;Age&quot;]],&quot;i9&quot;,&quot;i10&quot;,&quot;i11&quot;,false]"><d
iv jscontroller="sWGJ4b" jsaction="EEvAHc:yfX9oc;" jsname="WsjYwc" class="geS5n"><div class="z12JJ"><div class="M4DNQ"><div id="i9" class="HoXoMd D1wxyf RjsPE" role="heading" aria-level="3"><span class="M7eMe">Age</span></div><div class="gubaDc OIC90c RjsPE" id="i10"></div></div></div><d
iv jsmodel="CP1oW" data-params="%.@.[1846923513,&quot;Age&quot;,null,2,[[839337160,[[&quot;18-24&quot;,null,null,null,false],[&quot;25-30&quot;,null,null,null,false],[&quot;31-40&quot;,null,null,null,false],[&quot;41-50&quot;,null,null,null,false],[&quot;51&quot;,null,null,null,false],[&quot;&quot;,null,null,null,true]],false,null,null,null,null,null,false,null,[]]],null,null,null,null,null,null,[null,&quot;Age&quot;]],&quot;i9&quot;,&quot;i10&quot;,&quot;i11&quot;,false]"><d
iv jscontroller="sWGJ4b" jsaction="EEvAHc:yfX9oc;" jsname="WsjYwc" class="geS5n"><div class="z12JJ"><div class="M4DNQ"><div id="i9" class="HoXoMd D1wxyf RjsPE" role="heading" aria-level="3"><span class="M7eMe">Age</span></div><div class="gubaDc OIC90c RjsPE" id="i10"></div></div></div><d

Query: "Select the radio button with label 'Other' under the 'Age' section, then type '16'"
Thought:
- The query asks to select a radio button with the label 'Other' under the 'Age' section and then type '16'.
- The provided HTML snippet shows a structure that seems to be part of a form, but the snippet is incomplete and improperly formatted (e.g., "iv" instead of "div" at the beginning).
- There is no visible radio button or input field with the label 'Other' in the provided HTML snippet.
- The 'Age' section is identified with an id of "i9", but there are no radio buttons or options listed in the snippet.
- The action of typing '16' suggests that there should be a text input field associated with the 'Other' radio button, but this is not present in the snippet.
- Without the complete and properly formatted HTML, it is not possible to determine if the 'Other' radio button exists or where to type '16'.

Identifiable: No

HTML:
{context_str}
Query: {query_str}
Thought:
Identifiable: 
'''

In [41]:
DEFAULT_PLAYWRIGHT_PROMPT = '''
Your goal is to write Playwright Python code to answer queries.

Your answer must be a Python markdown only.

Prefer User-Facing Attributes, Use text selectors, like text="Visible Text", to target elements by their visible text. 
You can also use Attributes like aria-label, aria-labelledby, role, etc., to target elements.
When user-facing attributes are not available or sufficient, Prefer class names and IDs that are meaningful and unlikely to change. 
Avoid using automatically generated, framework-specific, or obfuscated classes.
Utilize parent-child relationships to narrow down the element, especially when looking for elements within a specific section of the page

You can assume the following code has been executed:
```python
from playwright.async_api import async_playwright

playwright = await async_playwright().start()
browser = await playwright.chromium.connect_over_cdp("http://localhost:9222")
default_context = browser.contexts[0]

# Retrieve the first page in the context.
page = default_context.pages[0]
```

---

HTML:
<!DOCTYPE html>
<html>
<head>
    <title>Mock Search Page</title>
</head>
<body>
    <h1>Search Page Example</h1>
    <input id="searchBar" type="text" placeholder="Type here to search...">
    <button id="searchButton">Search</button>
    <script>
        document.getElementById('searchButton').onclick = function() {{
            var searchText = document.getElementById('searchBar').value;
            alert("Searching for: " + searchText);
        }};
    </script>
</body>
</html>

Query: Click on the search bar 'Type here to search...', type 'selenium', and press the 'Enter' key

Completion:
```python
# Let's proceed step by step.
# First we need to identify the component first, then we can click on it.

# Based on the HTML, the link can be uniquely identified using the ID "searchBar"
# Click on the search bar
search_bar = page.locator('#searchBar')
search_bar.click()

# Type 'selenium' into the search bar
await page.type('#searchBar', 'selenium')

# Press the 'Enter' key
await page.keyboard.press('Enter')

```

---

HTML:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Mock Page for Selenium</title>
</head>
<body>
    <h1>Welcome to the Mock Page</h1>
    <div id="links">
        <a href="#link1" id="link1">Link 1</a>
        <br>
        <a href="#link2" class="link">Link 2</a>
        <br>
    </div>
</body>
</html>

Query: Click on the title Link 1 and then click on the title Link 2

Completion:
```python
# Let's proceed step by step.
# First we need to identify the first component, then we can click on it. Then we can identify the second component and click on it.

# Based on the HTML, the first link the link can be uniquely identified using the ID "link1"
# Let's use this ID with playwright to identify the link
link1 = page.locator('#link1')

# Then we click on the link
await link1.click()

# The other link can be uniquely identified using the class "link"
# Let's use this class to identify the link
link2 = page.locator('.link')

# Click on the element found
await link2.click()
```

---

HTML:
<!DOCTYPE html>
<html>
<head>
    <title>Mock Page</title>
</head>
<body>
    <p id="para1">This is the first paragraph.</p>
    <p id="para2">This is the second paragraph.</p>
    <p id="para3">This is the third paragraph, which we will select and copy.</p>
    <p id="para4">This is the fourth paragraph.</p>
</body>
</html>

Query: Select the text inside the third paragraph

Completion:
```python
# Let's proceed step by step.

# Select the third paragraph element
third_paragraph = page.locator("(//p)[3]")
# Get the text inside the third paragraph
text = third_paragraph.inner_text()
```

---

HTML:

Query: Scroll up a bit

Completion:
```python
# Let's proceed step by step.
# We don't need to use the HTML data as this is a stateless operation.
# 200 pixels should be sufficient. Let's execute the JavaScript to scroll up.

await page.evaluate("window.scrollBy(0, 200)")
```

---

---

HTML:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Enhanced Mock Page for Selenium Testing</title>
</head>
<body>
    <h1>Enhanced Test Page for Selenium</h1>
    <div class="container">
        <button id="firstButton" onclick="alert('First button clicked!');">First Button</button>
        <!-- This is the button we're targeting with the class name "action-btn" -->
        <button class="action-btn" onclick="alert('Action button clicked!');">Action Button</button>
        <div class="nested-container">
            <button id="testButton" onclick="alert('Test Button clicked!');">Test Button</button>
        </div>
        <button class="hidden" onclick="alert('Hidden button clicked!');">Hidden Button</button>
    </div>
</body>
</html>


Query: Click on the Button 'Action Button'

Completion:
```python
# Let's proceed step by step.
# First we need to identify the button first, then we can click on it.

# Based on the HTML provided, we need to devise the best strategy to select the button.
# The action button can be identified using the class name "action-btn"
action_button = page.locator('.action-btn')

# Then we can click on it
await action_button.click()
```

---

HTML:
{context_str}
Query: {query_str}
Completion:
'''

# Get the prompt

In [42]:
# from IPython.core.magic import register_line_magic, needs_local_scope
# import pyperclip
# import asyncio

# @register_line_magic
# @needs_local_scope
# def execute(line, local_ns=None):
#     # Your async function adapted for magic command
#     async def async_execute(line):
#         # Assuming 'page' is available in the local namespace
#         await gemini_page.goto("https://aistudio.google.com/app/prompts/new_chat")
        
#         input_bar = gemini_page.locator('[placeholder="Type something"]')
        
#         # Click on the input bar
#         await input_bar.click()

#         # Type the line into the input bar
#         await input_bar.type(line)
        
#         # Press Enter
#         await asyncio.sleep(1.5)                
        
#         await input_bar.press('Enter')
        
#         copy_to_clipboard_button = gemini_page.locator('button[mattooltip="Copy to clipboard"]')

#         # Then we can click on it
#         await copy_to_clipboard_button.click()
#         clip = pyperclip.paste()
#         print(clip)
    
#     # Run the async function using asyncio
#     loop = asyncio.get_event_loop()
#     if loop.is_running():  # For Jupyter notebooks, the loop is usually already running
#         task = loop.create_task(async_execute(line))
#         # Note: In Jupyter, you can't wait for the task directly as it blocks the notebook.
#         # You could potentially use widgets or other mechanisms to track completion.
#     else:
#         # This is for completeness and general knowledge. In practice, Jupyter's loop is always running.
#         loop.run_until_complete(async_execute(line))
        
# async def async_execute():
#     # Assuming 'page' is available in the local namespace
#     await gemini_page.goto("https://aistudio.google.com/app/prompts/new_chat")
    
#     input_bar = gemini_page.locator('[placeholder="Type something"]')
    
#     # Click on the input bar
    
#     await input_bar.press('Control+V')

#     # Type the line into the input bar
    
#     # Press Enter
#     await asyncio.sleep(1.5)                
    
#     await input_bar.press('Enter')
    
#     await gemini_page.locator('button[mattooltip="Copy to clipboard"]').first.click()
#     clip = pyperclip.paste()
#     return clip

# query = "Write a hello world program in Python"
# pyperclip.copy(query)
# loop = asyncio.get_event_loop()
# task = await loop.create_task(async_execute())

In [43]:


# code, source_nodes = remote_action_engine.get_nodes(query, html)
# context_str = source_nodes[0]
# prompt = DEFAULT_PLAYWRIGHT_PROMPT.format(context_str=beautify_html(context_str), query_str=query)
# pyperclip.copy(prompt)
# loop = asyncio.get_event_loop()
# code = await loop.create_task(async_execute())

# Carousel

In [44]:
# from IPython.display import display, HTML, Javascript




# total_nodes = ""
# for node in source_nodes:
#     html_code = beautify_html(node)
#     # print(html_code)
#     # print("-"*80)

#     escaped_html = html_code.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;')

#     # Now create a HTML format to display as "Markdown" within an HTML block
#     # We use the <pre> and <code> tags for proper formatting
#     display_html = f'<pre><code>{escaped_html}</code></pre>'
#     line = f"""<div class="carousel-item" style="min-width: 200px; height: 500px; background-color: #f99; text-align: center; line-height: 100px;">{display_html}</div>"""
#     total_nodes += line

# carousel_html = f"""
# <div style="width: 200px; overflow: hidden; margin: 0 auto;">
#   <div id="carouselExample" class="carousel" style="display: flex; overflow-y: hidden; scroll-behavior: smooth;">
#     {total_nodes}
#   </div>
#   <button onclick="scrollCarousel(-200)" style="position: absolute; top: 50%; left: 0; transform: translateY(-50%);">&#10094;</button>
# <button onclick="scrollCarousel(200)" style="position: absolute; top: 50%; right: 0; transform: translateY(-50%);">&#10095;</button>
# </div>
# """

# display(HTML(carousel_html))

# javascript = """
# <script type="text/Javascript">
#   function scrollCarousel(offset) {
#     var container = document.getElementById('carouselExample');
#     container.scrollLeft += offset;
#   }
# </script>
# """

# display(HTML(javascript))


# Google form

## Get identification result

In [29]:
context_str = "\n".join(source_nodes)

prompt = CLASSIFICATION_PROMPT.format(query_str=instruction, context_str=context_str)
output = llm.complete(prompt).text

print(output)

Thought:
- The query asks to select a radio button with the label 'Other' under the 'Age' section and then type '16'.
- The provided HTML does not show any radio buttons or an input field where text such as '16' could be entered.
- The 'Age' section is identified by the role="heading" and aria-level="3" with the id "i9", but there are no radio buttons or 'Other' label visible in the HTML snippet.
- There is no element with a placeholder or text that indicates an option for 'Other' or a field to enter a number like '16'.
- Without the appropriate input elements, the action specified in the query cannot be performed based on the given HTML.

Identifiable: No


In [40]:
input_field = page.locator('input[aria-labelledby="i1"]')

# Click on the inpu
outer_html = await input_field.evaluate("el => el.outerHTML") 
print(outer_html)

<input type="text" class="whsOnd zHQkBf" jsname="YPqjbf" autocomplete="off" tabindex="0" aria-labelledby="i1" aria-describedby="i2 i3" aria-disabled="false" required="" dir="auto" data-initial-dir="auto" data-initial-value="Daniel" badinput="false">


In [10]:
for output in outputs:
    print(output)

Thought:
- The query asks to select an input field with the placeholder 'Your answer' that is located below the 'Name' section and then type 'Daniel'.
- The 'Name' section is identified with an id of "i1", and the description for where to fill the name is identified with an id of "i2".
- The input field associated with the 'Name' section can be identified by the aria-labelledby attribute that references "i1" and the aria-describedby attribute that references "i2".
- The placeholder 'Your answer' is found within a div with the class "ndJi5d snByac" that is aria-hidden, which means it is meant for visual representation rather than being an actual placeholder attribute in the input field.
- The input field itself does not have a placeholder attribute, but the visual representation of the placeholder is present, which matches the user's query.

Identifiable: Yes
Thought:
- The task is to find an input field with the placeholder 'Your answer' that is located below the 'Phone number' label.


In [26]:
instructions

["Select the input with placeholder 'Your answer' below 'Name', then type 'Daniel'",
 "Select the input placeholder 'Your answer' below 'Phone number', then type '+123'",
 "Select the radio button with label 'Other' under the 'Age' section, then type '16'"]

In [56]:
from tqdm import tqdm
from IPython.display import display, Markdown, HTML, Code
import asyncio

page.set_default_timeout(5000)

instructions = """- Select the input with placeholder 'Your answer' below 'Name', then type 'Daniel'
- Select the input placeholder 'Your answer' below 'Phone number', then type '+123'
- Select the radio button with label 'Other' under the 'Age' section, then type '16'"""
instructions = instructions.split('\n')
instructions = [instruction.replace("- ", "") for instruction in instructions]

outputs = []

for instruction in tqdm(instructions):
    query = instruction
    html = await page.content()
    
    output = {}
    
    code, source_nodes, metadata = remote_action_engine.process_fat_node(query, html)
    
    retrieve_code = metadata['retrieve_code']
    indexing_time = metadata['indexing_time']
    completion_time = metadata['completion_time']
    model_id = metadata['model_id']
    prompt = metadata['prompt']
    
    context_str = "\n".join(source_nodes)
    
    print("Code generated:")
    
    # Executes async the generated code
    test_func = None
    indented_code = code.replace('\n', '\n\t')
    indented_code = f"async def test_func():\n\t{indented_code}"
    display(Code(indented_code, language='python'))
    
    # We execute the generated code
    exec(indented_code, globals())
    loop = asyncio.get_event_loop()
    if test_func:
        try:
            await loop.create_task(test_func())
            execution_success = True
            execution_error = ""
            identification_result = ""
        except Exception as e:
            print("Error occurred:", e)
            execution_success = False
            execution_error = str(e)
            
            identification_prompt = CLASSIFICATION_PROMPT.format(query_str=instruction, context_str=context_str)
            identification_result = llm.complete(identification_prompt).text
            
            for i, node in enumerate(source_nodes):
                display(Markdown(f"### Node {i+1}"))
                display(Code(beautify_html(node), language='html'))
                display(HTML(node))
    else:
        raise Exception("test_func not defined, issue generating code")
    output = {
        "html": html,
        "source_nodes": source_nodes,
        "query": query,
        "prompt": prompt,
        "code": code,
        "execution_success": execution_success,
        "execution_error": execution_error,
        "identification_result": identification_result,
        "model_id": model_id,
        "retrieve_code": retrieve_code,
        "indexing_time": indexing_time,
        "completion_time": completion_time
    }
    outputs.append(output)
    

  0%|          | 0/3 [00:00<?, ?it/s]

Code generated:


async def test_func():
	# Let's proceed step by step.
	# First we need to identify the input field below the 'Name' label, then we can type into it.
	
	# Based on the HTML provided, we can use the aria-labelledby attribute to find the input field associated with the 'Name' label.
	# The 'Name' label has an id of "i1", so we can use that to find the input field.
	name_input = page.locator('input[aria-labelledby="i1"]')
	
	# Click on the input field to focus on it before typing
	await name_input.click()
	
	# Type 'Daniel' into the input field
	await name_input.fill('Daniel')

 33%|███▎      | 1/3 [00:13<00:27, 13.62s/it]

Code generated:


async def test_func():
	# Let's proceed step by step.
	# First we need to identify the input field with the placeholder 'Your answer' that is below the 'Phone number' label.
	
	# We can use the aria-labelledby attribute to find the input field associated with the 'Phone number' label.
	# The 'Phone number' label has an id of "i5", so we will use that to find the input field.
	phone_number_input = page.locator('input[aria-labelledby="i5"]')
	
	# Click on the input field to focus on it
	await phone_number_input.click()
	
	# Type '+123' into the input field
	await phone_number_input.type('+123')

 67%|██████▋   | 2/3 [00:23<00:11, 11.63s/it]

Code generated:


async def test_func():
	# Let's proceed step by step.
	# First, we need to identify the radio button with the label 'Other' under the 'Age' section.
	
	# Based on the HTML structure, we can use the text selector to find the radio button with the label 'Other'.
	other_radio_button = page.locator('text="Other:"')
	
	# Click on the radio button
	await other_radio_button.click()
	
	# Now we need to type '16' into the input field that appears after selecting 'Other'.
	# We can use the aria-label attribute to find the input field.
	other_input_field = page.locator('input[aria-label="Other response"]')
	
	# Type '16' into the input field
	await other_input_field.fill('16')

100%|██████████| 3/3 [00:32<00:00, 10.84s/it]


In [57]:
import pandas as pd

df = pd.DataFrame(outputs)
df

,html,source_nodes,query,prompt,code,execution_success,execution_error,identification_result,model_id,retrieve_code,indexing_time,completion_time
0,"<!DOCTYPE html><html lang=""en"" class=""HB1eCd-U...","[<div class=""x0k1lc MbhUzd""></div><div class=""...",Select the input with placeholder 'Your answer...,\nYour goal is to write Playwright Python code...,# Let's proceed step by step.\n# First we need...,True,,,gpt-4,"def get_retriever_recursive(embed, html):\n ...",1.634843,11.401597
1,"<!DOCTYPE html><html lang=""en"" class=""HB1eCd-U...","[<div class=""Axnmtb""><div class=""foqfDc gdyQ3c...",Select the input placeholder 'Your answer' bel...,\nYour goal is to write Playwright Python code...,# Let's proceed step by step.\n# First we need...,True,,,gpt-4,"def get_retriever_recursive(embed, html):\n ...",0.365440,9.239383
2,"<!DOCTYPE html><html lang=""en"" class=""HB1eCd-U...","[<div class=""x0k1lc MbhUzd""></div><div class=""...",Select the radio button with label 'Other' und...,\nYour goal is to write Playwright Python code...,"# Let's proceed step by step.\n# First, we nee...",True,,,gpt-4,"def get_retriever_recursive(embed, html):\n ...",0.366535,7.687490


# Misc

In [24]:
from playwright.sync_api import ElementHandle

async def toggle_highlight(element: ElementHandle, highlight: bool):
    """Highlight or unhighlight the element on the page."""
    if highlight:
        # Apply CSS styles to highlight the element
        await element.evaluate('''(element) => {
            element.style.outline = "2px dashed blue";
            element.style.backgroundColor = "rgba(135, 206, 250, 0.3)";
            element.style.borderRadius = "10px";
        }''')
    else:
        # Reset CSS styles to remove the highlight
        await element.evaluate('''(element) => {
            element.style.outline = "";
            element.style.backgroundColor = "";
            element.style.borderRadius = "";
        }''')
    return True

In [25]:
await toggle_highlight(age_section, highlight=True)

True

In [14]:
outer_html = await other_radio_button.evaluate("el => el.outerHTML") 
print(beautify_html(outer_html))

<div aria-checked="true" aria-posinset="6" aria-setsize="6" class="Od2TWd hYsg7c N2RpBe" data-value="__other_option__" id="i28" jsaction="keydown:I481le;dyRcpb:dyRcpb;click:cOuCgd; mousedown:UX7yZ; mouseup:lbsD7e; mouseenter:tfO1Yc; mouseleave:JywGue; focus:AHmuwe; blur:O22p3e; contextmenu:mg9Pef;touchstart:p6p2H; touchmove:FwuNnf; touchend:yfqBxc(preventDefault=true); touchcancel:JMtRjd;" jscontroller="D8e5bc" jsshadow="" role="radio" tabindex="0">
 <div class="x0k1lc MbhUzd">
 </div>
 <div class="uyywbd">
 </div>
 <div class="vd3tt">
  <div class="AB7Lab Id5V1">
   <div class="rseUEf nQOrEb">
   </div>
  </div>
 </div>
</div>

